In [4]:
# %matplotlib nbagg
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import time
sys.path.append("..")
os.chdir("/home/nakamura/HAI/main_server/hai")

import coloredlogs, logging
logger = logging.getLogger(__name__)
coloredlogs.install(level='CRITICAL', logger=logger)

import pymongo
from pymongo import MongoClient
from PIL import Image
from matplotlib import pylab as plt
from matplotlib.animation import ArtistAnimation
import matplotlib.animation as animation
import numpy as np
from bson.objectid import ObjectId
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from notebooks.utils.utils import visualize, display_latest_image, display_image, print_time, strtime_to_epoch, epoch_to_strtime, UpdateDist, display_two_images
from controllers.actionrec import ActionRecognition
from controllers.dbreader.hue_koki_dbreader import HueDBReader, pair_images, extract_color, ExprHueDBReader
from controllers.dbreader.opedbreader import OperationDBReader, collect_img_tv, collect_img_music
from controllers.dbreader.imagereader import ImageReader
from controllers.vectorizer.posennvectorizer import NNVectorizer, ActVectorizer
from controllers.vectorizer.person2vec import Person2Vec

mongo = MongoClient('localhost', 20202).hai

from flask import Flask
app = Flask(__name__)
app.config.from_pyfile(filename="application.cfg")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [5]:
start = strtime_to_epoch("2017-12-17 14:00:00")
end = strtime_to_epoch("2017-12-17 15:05:00")

In [55]:
imreader = ImageReader()
X, y = imreader.read_db('koki', start, end, ['webcam0', 'webcam1'])

In [56]:
labels = []
for yi in y:
    if yi not in labels:
        labels.append(yi)
        
new_y = []
for yi in y:
    for i in range(len(labels)):
        if labels[i] == yi:
            new_y.append(i)
            break

In [58]:
labels

[({'bri': 254, 'hue': 14910, 'on': True, 'sat': 144}, 'stop_youtube', 'off'),
 ({'bri': 254, 'hue': 14910, 'on': True, 'sat': 144}, 'play_youtube', 'off'),
 ({'bri': 254, 'hue': 14910, 'on': True, 'sat': 144}, 'stop_youtube', 'on'),
 ({'bri': 254, 'hue': 2049, 'on': True, 'sat': 0}, 'stop_youtube', 'off'),
 ({'bri': 254, 'hue': 2049, 'on': True, 'sat': 0}, 'play_youtube', 'off'),
 ({'on': False}, 'play_youtube', 'off'),
 ({'on': False}, 'stop_youtube', 'off'),
 ({'bri': 254, 'hue': 11527, 'on': True, 'sat': 254}, 'play_youtube', 'off')]

In [62]:
X_train, X_val, y_train, y_val = train_test_split(X, new_y, train_size=0.6, random_state=1)
person2vectorizer = Person2Vec()
pose_arr_train, act_arr_train, meta = person2vectorizer.vectorize(X_train, get_meta=True)
pose_arr_val, act_arr_val, meta = person2vectorizer.vectorize(X_val, get_meta=True)

/home/nakamura/.pyenv/versions/anaconda3-4.4.0/envs/detection_server/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [63]:
clf = RandomForestClassifier()
# clf = linear_model.SGDClassifier(loss='log')
pipe = Pipeline([
            ("scale", StandardScaler()),
            ("pca", PCA(n_components=50)),
            ("clf", clf)
    ])
# cross_validation.cross_val_score(pipe, nn_mat_val, y_val, cv=5)
# X_train, X_val, y_train, y_val = train_test_split(X_arr, y_arr, train_size=0.6, random_state=1)
pipe.fit(pose_arr_train, y_train)
pipe.score(pose_arr_val, y_val)
# y_predicted = pipe.predict(jk)

0.85345997286295794

In [64]:
clf = RandomForestClassifier()
# clf = linear_model.SGDClassifier(loss='log')
pipe = Pipeline([
            ("scale", StandardScaler()),
            ("pca", PCA(n_components=50)),
            ("clf", clf)
    ])
pipe.fit(act_arr_train, y_train)
pipe.score(act_arr_val, y_val)

0.88195386702849388